# Model

In [1]:
import os
import sys

import pathlib

notebook_path = pathlib.Path(os.getcwd())
sys.path.append(str(notebook_path.parent))

In [2]:
import numpy as np
import pandas as pd

import pickle

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder
)
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import (
    cross_val_score,
    GridSearchCV
)

from IPython.display import HTML

from database import engine

## Loading data

- We only load records that belong to the `view` category, because every like starts with view, and we have a `target` column that marks those views that lead to likes.

In [3]:
user_data = pd.read_sql(
    f"SELECT * FROM public.user_data LIMIT 100;",
    con = engine,
    index_col = "user_id"
)
post_data = pd.read_sql(
    f"SELECT * FROM public.post_text_df LIMIT 100;",
    con = engine,
    index_col = "post_id"
)
feed_data = pd.read_sql(
    f"SELECT * FROM public.feed_data limit 100;",
    con = engine
)

Sometimes this frame loads too slowly, so I use a temporary local file that stores the data locally.

In [7]:
joined_data = pd.read_parquet("preloaded_joined_data.parquet")

In [8]:
df_show = {
    "Users data" : user_data,
    "Post data" : post_data,
    "Feed data" : feed_data,
    "Joined data" : joined_data
}

for title, df in df_show.items():
    display(HTML(f"<h3>{title}</h3>"))
    display(df.head())

,gender,age,country,city,exp_group,os,source
user_id,,,,,,,
200,1,34,Russia,Degtyarsk,3,Android,ads
201,0,37,Russia,Abakan,0,Android,ads
202,1,17,Russia,Smolensk,4,Android,ads
203,0,18,Russia,Moscow,1,iOS,ads
204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


,text,topic
post_id,,
1,UK economy facing major risks\n\nThe UK manufa...,business
2,Aids and climate top Davos agenda\n\nClimate c...,business
3,Asian quake hits European shares\n\nShares in ...,business
4,India power shares jump on debut\n\nShares in ...,business
5,Lacroix label bought by US firm\n\nLuxury good...,business


,timestamp,user_id,post_id,action,target
0,2021-12-23 09:24:22,113104,1901,view,1
1,2021-12-23 09:25:12,113104,1901,like,0
2,2021-12-23 09:25:14,113104,4962,view,0
3,2021-12-23 09:25:59,113104,4853,view,0
4,2021-12-23 09:26:50,113104,2665,view,1


,user_id,timestamp,target,gender,age,country,city,exp_group,os,source,text,topic
0,113104,2021-12-23 09:24:22,1,0,24,Russia,Rostov,4,Android,organic,Slovakia seal Hopman Cup success\n\nSlovakia c...,sport
1,113104,2021-12-23 09:28:44,1,0,24,Russia,Rostov,4,Android,organic,The first person to code a #COVID19 tracing ap...,covid
2,113104,2021-12-23 09:39:19,0,0,24,Russia,Rostov,4,Android,organic,Fox attacks Blairs Tory lies\n\nTony Blair lie...,politics
3,113104,2021-12-23 09:41:26,0,0,24,Russia,Rostov,4,Android,organic,Featured in 1955s THE COBWEB is an all star ca...,movie
4,113104,2021-12-23 09:44:02,0,0,24,Russia,Rostov,4,Android,organic,"I enjoyed Oceans 11, I thought it was quite en...",movie


## Transformation

In [16]:
X = joined_data.drop("user_id", axis = 1).copy()

y = X["target"]
X.drop("target", axis = 1, inplace = True)

X["month"] = X["timestamp"].dt.month
X["year"] = X["timestamp"].dt.year
X["hour"] = X["timestamp"].dt.hour
X.drop("timestamp", axis = 1, inplace = True)

X['gender'] = X['gender'].astype("O")

In [17]:
numeric_columns = list(X.select_dtypes("number").columns)
categorical_columns = [
    'gender', 'country', 'city', 'os', 'source', 'topic'
]
text_columns = 'text'

In [7]:
data_transformer = ColumnTransformer([
    ("stand_scaler", StandardScaler(), numeric_columns),
    (
        "one_hot_encod", 
        OneHotEncoder(
            categories = list(X[categorical_columns].apply(lambda col: list(col.unique())))
        ), 
        categorical_columns
    ),
    ("tf_idf_vector", TfidfVectorizer(max_features = 10), text_columns)
])

## Model pipeline

In [8]:
pipeline = Pipeline([
    ("transfmer", data_transformer),
    ("model", GradientBoostingClassifier())
])

In [9]:
param_grid = {
    "model__learning_rate" : [0.05, 0.1, 0.2],
    "model__max_depth" : np.arange(4, 20, 3)
}

grid_search_result = GridSearchCV(
    estimator = pipeline,
    param_grid = param_grid,
    verbose = 2
).fit(X, y)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] END ......model__learning_rate=0.05, model__max_depth=4; total time=   0.4s
[CV] END ......model__learning_rate=0.05, model__max_depth=4; total time=   0.5s
[CV] END ......model__learning_rate=0.05, model__max_depth=4; total time=   0.4s
[CV] END ......model__learning_rate=0.05, model__max_depth=4; total time=   0.4s
[CV] END ......model__learning_rate=0.05, model__max_depth=4; total time=   0.4s
[CV] END ......model__learning_rate=0.05, model__max_depth=7; total time=   0.6s
[CV] END ......model__learning_rate=0.05, model__max_depth=7; total time=   0.6s
[CV] END ......model__learning_rate=0.05, model__max_depth=7; total time=   0.6s
[CV] END ......model__learning_rate=0.05, model__max_depth=7; total time=   0.6s
[CV] END ......model__learning_rate=0.05, model__max_depth=7; total time=   0.6s
[CV] END .....model__learning_rate=0.05, model__max_depth=10; total time=   1.0s
[CV] END .....model__learning_rate=0.05, model__

In [10]:
best_model = pipeline.set_params(
    **grid_search_result.best_params_
).fit(X, y)

file_name = "model.pck"
pickle.dump(best_model, open(file_name, "wb"))

## Data for checker

I need to save precomputed data to the checker database - it needs features that I have generated for the model.

In [23]:
ready_features = X.copy()
# it needs features that my model will need and
# user_id column from original dataframe
ready_features["user_id"] = joined_data["user_id"]
ready_features.to_sql(
    con = engine,
    name = "fedor_kobak_features_lesson_22",
    if_exists = "replace"
)